In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pathlib import Path
import math
from sklearn.metrics import mean_squared_error

In [ ]:
DATA_FILE = "ffmpeg-results-lottermann.csv"

SA_KEY = "sa"
TA_KEY = "ta"
QP_KEY = "qp"
RATE_KEY = "rate"
GOP_KEY = "gop_len"
BITRATE_KEY = "bitrate"

In [ ]:
# Lottermann Bitrate model
QP_min  = 24
F_max   = 30         # Hz
RHO_1   = 1.3190     # kbit/s
RHO_2   = 31.7481    # kbit/s
RHO_3   = -210.6505  # kbit/s
alpha_1 = -0.0492
alpha_2 = 4.8648
alpha_3 = -14.3861
b       = 1
d       = 1
gamma_1 = -0.0108
gamma_2 = 0.9473
eps_1   = 0.0011
eps_2   = 1.0200
eps_3   = -0.0598

def bitrate_model(SA, TA, QP, f, n):
    R_max = RHO_1 * TA * SA + RHO_2 * SA + RHO_3
    a = alpha_1 * SA + alpha_2 * math.log(SA) + alpha_3
    SCF = pow(QP / QP_min, -a)
    TCF = pow(f / F_max, b)
    c = gamma_1 * TA + gamma_2
    e = eps_1 * SA + eps_2 * (TA / SA) + eps_3
    NCF = c * pow(1 / n, d) + e
    MCF = 1 # P-frames only

    return R_max * SCF * TCF * NCF * MCF

In [ ]:
def get_bitrates_for_video(df, video_key):
    video_data = df.loc[video_key]
    bitrate_measured = (video_data[BITRATE_KEY] * 0.001).tolist()
    bitrate_estimated = [bitrate_model(sa, ta, qp, f, n) for sa, ta, qp, f, n in zip(
        video_data[SA_KEY].tolist(), 
        video_data[TA_KEY].tolist(),
        video_data[QP_KEY].tolist(),
        video_data[RATE_KEY].tolist(),
        video_data[GOP_KEY].tolist()
    )]
    return bitrate_measured, bitrate_estimated

def get_rmse(bitrate_measured, bitrate_estimated):
    return math.sqrt(mean_squared_error(bitrate_measured, bitrate_estimated))

def get_pc(bitrate_measured, bitrate_estimated):
    return np.corrcoef(bitrate_measured, bitrate_estimated)[0,1]

In [ ]:
df = pd.read_csv(DATA_FILE, index_col="name")

In [ ]:
# Visu SA/TA
video_keys = df.index.drop_duplicates()
unique_rows = [df.loc[key].head(1) for key in video_keys]
ax = plt.gca()
df.plot.scatter(x=TA_KEY, y=SA_KEY, ax=ax)
for it in unique_rows:
    ax.annotate(it.index[0], xy=(it["ta"][0], it["sa"][0]))

plt.xlabel("TA")
plt.ylabel("SA")
plt.title("TA/SA")


In [ ]:
video_keys = df.index.drop_duplicates()
for key in video_keys:
    bitrate_measured, bitrate_estimated = get_bitrates_for_video(df, key)
    rmse = get_rmse(bitrate_measured, bitrate_estimated)
    pc = get_pc(bitrate_measured, bitrate_estimated)
    plt.figure()
    plt.plot(bitrate_estimated, bitrate_measured)
    plt.xlabel("Estimated Bitrate [kBit/s]")
    plt.ylabel("Measured Bitrate [kBit/s]")
    plt.title(f"Bitrate model comparison {key} (RMSE: {rmse:.2f}, PC: {pc:.5f})")
    print(f"RMSE {key}: {rmse:.2f}")
    print(f"PC {key}: {pc:.5f}")